In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model

In [2]:
df = yf.download('^GSPC', start = '2022-01-01', end = '2025-09-20')

df['returns'] = np.log(df['Close'] / df['Close'].shift(1)) # log returns
returns = df['returns'].dropna()
returns

/tmp/ipykernel_235/3119577417.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('^GSPC', start = '2022-01-01', end = '2025-09-20')
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Date
2022-01-04   -0.000630
2022-01-05   -0.019583
2022-01-06   -0.000964
2022-01-07   -0.004058
2022-01-10   -0.001442
                ...   
2025-09-15    0.004696
2025-09-16   -0.001289
2025-09-17   -0.000971
2025-09-18    0.004778
2025-09-19    0.004874
Name: returns, Length: 931, dtype: float64

### GARCH Model

In [3]:
model = arch_model(returns, vol = 'GARCH', p = 1, q = 1, rescale=False) # Call the mode
results = model.fit(disp = 'off') # Train the model

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                2962.90
Distribution:                  Normal   AIC:                          -5917.80
Method:            Maximum Likelihood   BIC:                          -5898.45
                                        No. Observations:                  931
Date:                Wed, Nov 05 2025   Df Residuals:                      930
Time:                        13:39:38   Df Model:                            1
                                 Mean Model                                 
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         8.2579e-04  3.461e-06    238.612      0.000 [8.190e-04,8.326e-04]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      2.6130e-06  2.281e-11  1.146e+05      0.000 [2.613e-06,2.613e-06]
alpha[1]       0.1000  2.267e-02      4.411  1.029e-05   [5.557e-02,  0.144]
beta[1]        0.8800  1.831e-02     48.051      0.000     [  0.844,  0.916]
============================================================================

Covariance estimator: robust
"""

In [4]:
#forcast daily 5 days volatility
forecast = results.forecast(horizon=5)

predicted_variance = forecast.variance.values[-1, :]
predicted_volatility = np.sqrt(predicted_variance)

print(predicted_volatility)

[0.00616058 0.00630925 0.00645163 0.00658817 0.0067193 ]


In [5]:
#adding variance to calcuate vol
predicted_var_5d = np.sum(predicted_volatility ** 2)    
predicted_vol_5d_garch = np.sqrt(predicted_var_5d)  
predicted_vol_5d_garch = predicted_vol_5d_garch * 100   #for scaling
print(predicted_vol_5d_garch)

1.4419979747598528


In [6]:
#Next 5 days returns
real_df = yf.download('^GSPC', start= '2025-09-19', end= '2025-09-27')
real_df['returns'] = np.log(real_df['Close'] / real_df['Close'].shift(1))
real_df = real_df.dropna()
real_df = real_df['returns']
real_df

/tmp/ipykernel_235/1830071241.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  real_df = yf.download('^GSPC', start= '2025-09-19', end= '2025-09-27')
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Date
2025-09-22    0.004400
2025-09-23   -0.005517
2025-09-24   -0.002851
2025-09-25   -0.005022
2025-09-26    0.005884
Name: returns, dtype: float64

In [7]:
# realized 5-day volatility
realized_var_5d = np.sum(real_df ** 2)    
realized_vol_5d = np.sqrt(realized_var_5d)  
realized_vol_5d = realized_vol_5d * 100 #scaling
print(realized_vol_5d)

1.0852409241040168


In [8]:
print("GARCH Model Predicted Volatility:", predicted_vol_5d_garch)
print("Realized Actual Volatility:", realized_vol_5d)

GARCH Model Predicted Volatility: 1.4419979747598528
Realized Actual Volatility: 1.0852409241040168


### EGARCH Model

In [9]:
#fit with EGARCH Model
model = arch_model(returns, mean='Zero', vol='EGARCH', p=1, q=1, dist='t', rescale=False)
results = model.fit(disp = 'off')
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Zero Mean - EGARCH Model Results                          
====================================================================================
Dep. Variable:                      returns   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.001
Vol Model:                           EGARCH   Log-Likelihood:                2972.50
Distribution:      Standardized Student's t   AIC:                          -5937.01
Method:                  Maximum Likelihood   BIC:                          -5917.66
                                              No. Observations:                  931
Date:                      Wed, Nov 05 2025   Df Residuals:                      931
Time:                              13:39:38   Df Model:                            0
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega         -0.1481  7.635e-02     -1.940  5.234e-02 [ -0.298,1.500e-03]
alpha[1]       0.1826  4.198e-02      4.350  1.363e-05   [  0.100,  0.265]
beta[1]        0.9833  8.382e-03    117.310      0.000   [  0.967,  1.000]
                              Distribution                              
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
nu             7.8535      1.986      3.955  7.644e-05 [  3.962, 11.745]
========================================================================

Covariance estimator: robust
"""

In [10]:
#forcast daily 5 days volatility
forecast = results.forecast(
    horizon=5,
    method="bootstrap",       
    simulations=2000,         
    random_state=np.random.RandomState(42),
    reindex=False             
)

predicted_variance = forecast.variance.values[-1, :]  
predicted_volatility = np.sqrt(predicted_variance)
print(predicted_volatility)

[0.00619773 0.00626612 0.00632837 0.00638871 0.00646942]


In [11]:
#adding variance to calcuate volatility
predicted_var_5d = np.sum(predicted_volatility ** 2)    
predicted_vol_5d_egarch = np.sqrt(predicted_var_5d)  
predicted_vol_5d_egarch = predicted_vol_5d_egarch * 100   #for scaling
print(predicted_vol_5d_egarch)

1.4156041410614022


In [12]:
print("EGARCH Model Predicted Volatility:", predicted_vol_5d_egarch)
print("Realized Actual Volatility:", realized_vol_5d)

EGARCH Model Predicted Volatility: 1.4156041410614022
Realized Actual Volatility: 1.0852409241040168


### GJR-GARCH Model

In [13]:
model = arch_model(returns, mean='Zero', vol='GARCH', p=1, o=1, q=1, dist='t', rescale=False)
results = model.fit(disp = 'off')
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Zero Mean - GJR-GARCH Model Results                         
====================================================================================
Dep. Variable:                      returns   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.001
Vol Model:                        GJR-GARCH   Log-Likelihood:                2994.46
Distribution:      Standardized Student's t   AIC:                          -5978.93
Method:                  Maximum Likelihood   BIC:                          -5954.75
                                              No. Observations:                  931
Date:                      Wed, Nov 05 2025   Df Residuals:                      931
Time:                              13:39:39   Df Model:                            0
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      2.4598e-06  4.082e-09    602.537      0.000  [2.452e-06,2.468e-06]
alpha[1]   1.1252e-11  3.211e-02  3.504e-10      1.000 [-6.293e-02,6.293e-02]
gamma[1]       0.1841  3.783e-02      4.867  1.133e-06      [  0.110,  0.258]
beta[1]        0.8921  2.643e-02     33.746 1.213e-249      [  0.840,  0.944]
                              Distribution                              
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
nu            10.2568      0.470     21.830 1.201e-105 [  9.336, 11.178]
========================================================================

Covariance estimator: robust
"""

In [14]:
#forcast daily 5 days volatility
forecast = results.forecast(
    horizon=5,
    method="bootstrap",       
    simulations=2000,         
    random_state=np.random.RandomState(42),
    reindex=False             
)

predicted_variance = forecast.variance.values[-1, :]  
predicted_volatility = np.sqrt(predicted_variance)
print(predicted_volatility)

[0.0054826  0.00566948 0.00582861 0.00597507 0.0061792 ]


In [15]:
#adding variance to calcuate volatility
predicted_var_5d = np.sum(predicted_volatility ** 2)    
predicted_vol_5d_gjrgarch = np.sqrt(predicted_var_5d)  
predicted_vol_5d_gjrgarch = predicted_vol_5d_gjrgarch * 100   #for scaling
print(predicted_vol_5d_gjrgarch)

1.304064656877277


In [16]:
print("GARCH Model Predicted Volatility:", predicted_vol_5d_garch)
print("EGARCH Model Predicted Volatility:", predicted_vol_5d_egarch)
print("GJR-GARCH Model Predicted Volatility:", predicted_vol_5d_gjrgarch)
print("Realized Actual Volatility:", realized_vol_5d)

GARCH Model Predicted Volatility: 1.4419979747598528
EGARCH Model Predicted Volatility: 1.4156041410614022
GJR-GARCH Model Predicted Volatility: 1.304064656877277
Realized Actual Volatility: 1.0852409241040168
